In [14]:
!pip install statsmodels

You should consider upgrading via the 'pip install --upgrade pip' command.


In [24]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.tsa.statespace.sarimax import SARIMAX
#from sklearn.metrics import mean_squared_error, r2_score